# Lectura de datos

In [8]:
import pandas as pd
from sklearn.feature_selection import mutual_info_regression,mutual_info_classif
import numpy as np
import json
FILE = 'data'

Leamos los datos para la métrica de información mutua, tratando la variable clase como una variable numérica.

In [9]:
df_numeric_class = pd.read_csv('%s/winequality-white.csv' %FILE, sep=";")
df_numeric_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


En el caso de la métrica CFS, transformamos los datos para convertir al atributo clase en uno nominal

In [17]:
df_nominal_class = df_numeric_class.copy()
df_nominal_class['quality'] = df_numeric_class['quality'].astype(str)
df_nominal_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   object 
dtypes: float64(11), object(1)
memory usage: 459.3+ KB


La clase nodo almacenará los estados de las combinaciones posibles  junto a su valor de correlación

In [11]:
class Node:
    def __init__(self, columns: list, value: float):
        self.columns = columns
        self.value   = value
    
    def __eq__(self, other):
        return self.value == other.value

    def __ne__(self, other):
        return self.value != other.value

    def __lt__(self, other):
        return self.value < other.value

    def __le__(self, other):
        return self.value <= other.value

    def __gt__(self, other):
        return self.value > other.value

    def __ge__(self, other):
        return self.value >= other.value
    
    def __repr__(self):
        return "%s = %s" % (self.columns, self.value)

El método *get_children* se encarga de a partir de un estado crear los N estados siguientes siguiendo esa rama del árbol de búsqueda, eligiendo las combinaciones faltantes. Se escoge como valor que guía la búsqueda la suma de todas las correlaciones.

In [12]:
def get_children(father: Node, columns: list, metric: classmethod, df: pd.DataFrame) -> list:
    if not father:
        return [
            Node([column], metric(df.loc[:,[column]], df.iloc[:,-1])[0]) for column in columns
        ]

    children = []
    for column in columns:
        if column not in father.columns:
            copyFatherColumns = father.columns.copy()
            copyFatherColumns.append(column)
            children.append(
                Node(copyFatherColumns,  sum(abs(metric(df.loc[:,copyFatherColumns], df.iloc[:,-1]))))
            )

    return children

*select_from_variables* se encarga de la exploración del árbol de estados mediante SFS y la métrica correspondiente.

In [13]:
def select_from_variables(df: pd.DataFrame, metric: classmethod, tree_deep: int = 4) -> pd.DataFrame:
    columns = list(df.columns[:-1])
    if len(columns) <= tree_deep:
        raise RuntimeError("the deep of the search tree is higher than the number of columns")
    """selection = []
    
    for _ in range(tree_deep):
        correlation_values = metric(df.loc[:,columns], df.iloc[:,-1])
        print("#############")
        print(sorted(correlation_values))
        column_select = columns[np.argmax(abs(correlation_values), axis=0)]
        columns.remove(column_select)
        selection.append(column_select)"""

    selection = None
    for _ in range(tree_deep):
        selection = max(get_children(selection, columns, mutual_info_regression, df))

    return selection
     

Para comparar entre las métricas se varia el número de variables seleccionadas de 2 en 2 hasta 8.

## Busqueda mediante regresión mutua

In [14]:
reportMIFS = {} 
for deep in range(2,10,2):
    reportMIFS['Tree Deep %s' %deep] = select_from_variables(df_numeric_class, mutual_info_regression, tree_deep=deep)

## Busqueda mediante clasificación mutua

In [15]:
reportCFS = {} 
for deep in range(2,10,2):
    reportCFS['Tree Deep %s' %deep] = select_from_variables(df_nominal_class, mutual_info_classif, tree_deep=deep)

# Conclusiones

In [16]:
for key, value in reportMIFS.items():
    print("#############",key,"#############")
    print("MIFS metric: ", sorted(value.columns))
print("CFS metric: ", sorted(reportCFS[key].columns))
    print("\n")

############# Tree Deep 2 #############
MIFS metric:  ['alcohol', 'density']
CFS metric:  ['alcohol', 'density']


############# Tree Deep 4 #############
MIFS metric:  ['alcohol', 'density', 'residual sugar', 'total sulfur dioxide']
CFS metric:  ['alcohol', 'density', 'residual sugar', 'total sulfur dioxide']


############# Tree Deep 6 #############
MIFS metric:  ['alcohol', 'chlorides', 'density', 'residual sugar', 'total sulfur dioxide', 'volatile acidity']
CFS metric:  ['alcohol', 'chlorides', 'density', 'free sulfur dioxide', 'residual sugar', 'total sulfur dioxide']


############# Tree Deep 8 #############
MIFS metric:  ['alcohol', 'chlorides', 'density', 'free sulfur dioxide', 'pH', 'residual sugar', 'total sulfur dioxide', 'volatile acidity']
CFS metric:  ['alcohol', 'citric acid', 'density', 'free sulfur dioxide', 'residual sugar', 'sulphates', 'total sulfur dioxide', 'volatile acidity']




En suma, se puede apreciar que para un número pequeño de atributos las dos métricas bajo un mismo método escogen los mismo atributos que son los que mayor correlación tienen con la variable clase, sin embargo cuando aumentamos el número de los mismo observamos que escogen atributos distintos. Mediante la métrica del MIFS se esconge las columnas de **ph** o **chlorides**, en cambio para la CFS se prefiere escoger las columnas de **citric acid** o **sulphates**. Solo faltaría probar los diferentes conjuntos en un modelo para comparar su calidad con respecto al problema.